Importing important libraries and the dataset obtained in the last exercise

In [2]:
import requests
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
loc_dataset=pd.read_csv('loc_dataset.csv')


Solving environment: done

# All requested packages already installed.



Plotting the map without clustering just for visualization

In [50]:
# create map of Toronto using latitude and longitude values
latitude=43.6532
longitude=-79.3832
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(loc_dataset['Latitude'], loc_dataset['Longitude'], loc_dataset['Borough'], loc_dataset['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto


Exploring the neighbourhood venues using FourSquare API and saving the details in a data frame

In [51]:
CLIENT_ID='IUO0GG4PTHSGPOSQRJ53FXWMLMKLXX0UOMEXUIDJATUJS43Q'
CLIENT_SECRET='HJUDRPVCDGOZIPGFNJUHH2LNNTEN4AOXPSLHNXWMPUDECTLQ'
VERSION = '20180605'
LIMIT=100
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)


In [52]:
toronto_venues = getNearbyVenues(names=loc_dataset['Neighbourhood'],
                                   latitudes=loc_dataset['Latitude'],
                                   longitudes=loc_dataset['Longitude']
                                  )
toronto_venues.head(20)

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Willowdale South,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Willowdale South,43.753259,-79.329656,KFC,43.754387,-79.333021,Fast Food Restaurant
2,Willowdale South,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,"CN Tower,Bathurst Quay,Island airport,Harbourf...",43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop
5,"CN Tower,Bathurst Quay,Island airport,Harbourf...",43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant
6,"CN Tower,Bathurst Quay,Island airport,Harbourf...",43.725882,-79.315572,Eglinton Ave E & Sloane Ave/Bermondsey Rd,43.726086,-79.313620,Intersection
7,Canada Post Gateway Processing Centre,43.654260,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
8,Canada Post Gateway Processing Centre,43.654260,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
9,Canada Post Gateway Processing Centre,43.654260,-79.360636,Toronto Cooper Koo Family Cherry St YMCA Centre,43.653191,-79.357947,Gym / Fitness Center


In [53]:
print(toronto_venues.shape)
toronto_venues.groupby('Neighbourhood').count()
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

(2231, 7)
There are 274 uniques categories.


In [54]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

,Neighbourhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide,King,Richmond",0.0,0.000000,0.000000,0.0000,0.0000,0.000,0.0000,0.000,0.050000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.025000
1,Agincourt,0.0,0.000000,0.000000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.00,0.000000,0.030303,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",0.0,0.000000,0.000000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",0.0,0.000000,0.000000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
4,"Alderwood,Long Branch",0.0,0.000000,0.000000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
5,"Bathurst Manor,Downsview North,Wilson Heights",0.0,0.000000,0.000000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
6,Bayview Village,0.0,0.000000,0.000000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
7,"Bedford Park,Lawrence Manor East",0.0,0.000000,0.000000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
8,Berczy Park,0.0,0.000000,0.000000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
9,"Birch Cliff,Cliffside West",0.0,0.000000,0.000000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.00,0.017857,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000


Figuring out the 10 most common venues for every neighbourhood

In [55]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Café,Coffee Shop,Gastropub,Italian Restaurant,Bakery,American Restaurant,Yoga Studio,Convenience Store,Brewery,Seafood Restaurant
1,Agincourt,Café,Restaurant,Bookstore,Japanese Restaurant,Bar,Bakery,College Arts Building,French Restaurant,Sandwich Place,Beer Store
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",Grocery Store,Fast Food Restaurant,Pizza Place,Sandwich Place,Beer Store,Coffee Shop,Fried Chicken Joint,Pharmacy,Electronics Store,Empanada Restaurant
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",Locksmith,Baseball Field,Business Service,Pool,Yoga Studio,Eastern European Restaurant,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
4,"Alderwood,Long Branch",Trail,Field,Dog Run,Hockey Arena,Yoga Studio,Dive Bar,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant


Setting up the kMeans clustering

In [56]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([4, 4, 4, 4, 4, 0, 1, 4, 4, 4], dtype=int32)

Merging loc_dataset with the neighborhoods_venues_sorted dataframe and adding the cluster labels

In [57]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = loc_dataset

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')


toronto_merged.head() # check the last columns!

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Willowdale South,43.753259,-79.329656,3.0,Fast Food Restaurant,Park,Food & Drink Shop,Dumpling Restaurant,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore
1,M4A,North York,"CN Tower,Bathurst Quay,Island airport,Harbourf...",43.725882,-79.315572,4.0,Intersection,Coffee Shop,Hockey Arena,Portuguese Restaurant,Dumpling Restaurant,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop
2,M5A,Downtown Toronto,Canada Post Gateway Processing Centre,43.654260,-79.360636,4.0,Coffee Shop,Park,Bakery,Café,Mexican Restaurant,Pub,Breakfast Spot,Electronics Store,Performing Arts Venue,Spa
3,M6A,North York,"Humber Bay Shores,Mimico South,New Toronto",43.718518,-79.464763,4.0,Furniture / Home Store,Clothing Store,Accessories Store,Arts & Crafts Store,Fraternity House,Event Space,Miscellaneous Shop,Coffee Shop,Boutique,Women's Store
4,M7A,Queen's Park,"East Birchmount Park,Ionview,Kennedy Park",43.662301,-79.389494,4.0,Coffee Shop,Diner,Gym,Park,Burrito Place,Italian Restaurant,Japanese Restaurant,Spa,Fast Food Restaurant,Seafood Restaurant


Plotting the map to show the clusters

In [71]:
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    if (cluster!=cluster)==False:
        {
        folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
}
        
    #else:
        #cluster=0    
           
map_clusters

The 4th cluster has the maximum number of neighbourhoods